## 変化検出およびコヒーレンス検出

SAR画像の取り出しについてはSAR_get.ipynbを参照してください

### 必要なライブラリの呼び出しと対象のSAR画像のtarget_idを定義

In [ ]:
from scipy.optimize import leastsq
import scipy, scipy.fftpack
import cv2
import matplotlib.pyplot as plt
from numpy import inf
import numpy as np

target_id1 = 'SAR_tokyo/ALOS2267472890-190507'
target_id2 = 'SAR_tokyo/ALOS2250912890-190115'

### 変化検出

In [ ]:
def ripoc(a, b, m = None):
    g_a = np.asarray(cv2.cvtColor(a, cv2.COLOR_BGR2GRAY), 'float')
    g_b = np.asarray(cv2.cvtColor(b, cv2.COLOR_BGR2GRAY), 'float')

    h, w = g_a.shape
    hy = np.hanning(h)
    hx = np.hanning(w)
    hw = hy.reshape(h, 1)*hx
    
    f_a = np.fft.fftshift(np.log(np.abs(np.fft.fft2(g_a*hw))))
    f_b = np.fft.fftshift(np.log(np.abs(np.fft.fft2(g_b*hw))))

    if not m:
        l = np.sqrt(w*w + h*h)
        m = l/np.log(l)

    center = (w/2, h/2)
    flags = cv2.INTER_LANCZOS4 + cv2.WARP_POLAR_LOG
    p_a = cv2.warpPolar(f_a, (w, h), center, m, flags)
    p_b = cv2.warpPolar(f_b, (w, h), center, m, flags)
    (x, y), e = cv2.phaseCorrelate(p_a, p_b, hw)

    angle = y*360/h
    scale = (np.e)**(x/m)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    t_b = cv2.warpAffine((g_b), M, (w, h))
    (x, y), e = cv2.phaseCorrelate(g_a, t_b)

    return x, y, angle, scale

def scatter_normalize(data):
    data = 10*np.log10(data) -83.0 -32.0
    data = np.array(255*(data-np.amin(data))/(np.amax(data)-np.amin(data)),dtype="uint8")
    return data

def make_image(HH_path,HV_path,VV_path,flag):
    Shh = scatter_normalize(np.load(HH_path))
    Shv = scatter_normalize(np.load(HV_path))
    Svv = scatter_normalize(np.load(VV_path))
    img=np.zeros((2000, 2000, 3), np.uint8)
    img[:,:,0]=Shh
    img[:,:,1]=Shv
    img[:,:,2]=Svv
    return cv2.flip(img, flag)

def registration(img1,img2,flag):
    # 位置合わせをしなかったとき
    if flag==0:
        sigma = np.array(img1,dtype=float)-np.array(img2,dtype=float)
        img = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
        plt.imshow(img)
        plt.imsave('raw_change.png', img)
    # 位置合わせを行うとき
    if flag==1:
        x, y, angle, scale = ripoc(img1, img2)
        print(x, y, angle, scale)

        h, w, ch = img1.shape
        M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
        M[0][2] -= x
        M[1][2] -= y
        dst = cv2.warpAffine(img2, M, (w, h))
        
        sigma = np.array(img1,dtype=float)-np.array(dst,dtype=float)
        img = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
        plt.imshow(img)
        plt.imsave('ripoc_change.png', img)
        return M, h, w
        
img1 = make_image('/home/jovyan/work/'+ target_id1 + '/HH.npy',
            '/home/jovyan/work/'+ target_id1 + '/HV.npy',
            '/home/jovyan/work/'+ target_id1 + '/VV.npy',
            1)
img2 = make_image('/home/jovyan/work/'+ target_id2 + '/HH.npy',
            '/home/jovyan/work/'+ target_id2 + '/HV.npy',
            '/home/jovyan/work/'+ target_id2 + '/VV.npy',
            1)
registration(img1,img2,0)

# このパラメータを使ってSARの生データの位置合わせを行います。
M, h, w = registration(img1,img2,1)

### SAR画像(RAWデータ)の位置合わせ

In [ ]:
def affine(data, M, h, w):
    # 8bitに正規化しないとRIPOCが上手く行かないので、
    # 正規化した方で変換パラメータだけ取得して、生データに対して変換パラメータを適応させる
    result = np.zeros((np.shape(data)[0],np.shape(data)[1]), dtype=np.complex)
    result.real = cv2.warpAffine(data.real, M, (w, h))
    result.imag = cv2.warpAffine(data.imag, M, (w, h))
    return result

def make_raw_image(HH,HV,VV,flag):
    Shh = 10*np.log10(HH) -83.0 -32.0
    Shv = 10*np.log10(HV) -83.0 -32.0
    Svv = 10*np.log10(VV) -83.0 -32.0
    img=np.zeros((2000, 2000, 3), np.uint8)
    img[:,:,0]=Shh
    img[:,:,1]=Shv
    img[:,:,2]=Svv
    return cv2.flip(img, flag)

# SARのRAWデータの読み込み
HH1 = np.load('/home/jovyan/work/'+ target_id1 + '/HH.npy')
HV1 = np.load('/home/jovyan/work/'+ target_id1 + '/HV.npy')
VV1 = np.load('/home/jovyan/work/'+ target_id1 + '/VV.npy')
HH2 = np.load('/home/jovyan/work/'+ target_id2 + '/HH.npy')
HV2 = np.load('/home/jovyan/work/'+ target_id2 + '/HV.npy')
VV2 = np.load('/home/jovyan/work/'+ target_id2 + '/VV.npy')

# slaveの生のSAR画像の位置合わせを行う
t_HH2 = affine(HH2, M, h, w)
t_HV2 = affine(HV2, M, h, w)
t_VV2 = affine(VV2, M, h, w)

master = make_raw_image(HH1,HV1,VV1,1)
slave = make_raw_image(t_HH2,t_HV2,t_VV2,1)

sigma = np.array(master,dtype=float)-np.array(slave,dtype=float)
img = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
plt.imshow(img)
plt.imsave('change_SAR_complex.png', img)

In [ ]:
# SARの生データのレジストレーション
HH1 = np.load('/home/jovyan/work/'+ target_id1 + '/HH.npy')
HV1 = np.load('/home/jovyan/work/'+ target_id1 + '/HV.npy')
VV1 = np.load('/home/jovyan/work/'+ target_id1 + '/VV.npy')
HH2 = np.load('/home/jovyan/work/'+ target_id2 + '/HH.npy')
HV2 = np.load('/home/jovyan/work/'+ target_id2 + '/HV.npy')
VV2 = np.load('/home/jovyan/work/'+ target_id2 + '/VV.npy')

def affine(data, M, h, w):
    # 8bitに正規化しないとRIPOCが上手く行かないので、
    # 正規化した方で変換パラメータだけ取得して、生データに対して変換パラメータを適応させる
    result = np.zeros((np.shape(data)[0],np.shape(data)[1]), dtype=np.complex)
    result.real = cv2.warpAffine(data.real, M, (w, h))
    result.imag = cv2.warpAffine(data.imag, M, (w, h))
    return result

# slaveの生のSAR画像の位置合わせを行う
t_HH2 = affine(HH2, M, h, w)
t_HV2 = affine(HV2, M, h, w)
t_VV2 = affine(VV2, M, h, w)

#flag = 0: 上下反転
#flag > 0: 左右反転
#flag < 0: 上下左右反転
def make_raw_image(HH,HV,VV,flag):
    Shh = 10*np.log10(HH) -83.0 -32.0
    Shv = 10*np.log10(HV) -83.0 -32.0
    Svv = 10*np.log10(VV) -83.0 -32.0
    img=np.zeros((2000, 2000, 3), np.uint8)
    img[:,:,0]=Shh
    img[:,:,1]=Shv
    img[:,:,2]=Svv
    return cv2.flip(img, flag)

master = make_raw_image(HH1,HV1,VV1,1)
slave = make_raw_image(t_HH2,t_HV2,t_VV2,1)

sigma = np.array(master,dtype=float)-np.array(slave,dtype=float)
# 一番最後に8bitの正規化を行う(どちらにせよ、これをやらないとエラーになる)
img = np.array(255*(sigma-np.amin(sigma))/(np.amax(sigma)-np.amin(sigma)),dtype="uint8")
plt.imshow(img)
plt.imsave('change_SAR_complex.png', img)

### 相関最大化アルゴリズムの適用

In [ ]:
def Complex_coherence(Shh1,Shv1,Svv1,Shh2,Shv2,Svv2,x,y):
    
    T11 = np.zeros((x,y,3,3), dtype=np.complex)
    T22 = np.zeros((x,y,3,3), dtype=np.complex)
    Omega12 = np.zeros((x,y,3,3), dtype=np.complex)
    
    for i in range(x):
        for j in range(y):
            k11 = Shh1[i,j]+Svv1[i,j]
            k12 = Svv1[i,j]-Shh1[i,j]
            k13 = 2*Shv1[i,j]
            k21 = Shh2[i,j]+Svv2[i,j]
            k22 = Svv2[i,j]-Shh2[i,j]
            k23 = 2*Shv2[i,j]
            
            k1 = np.array([k11, k12, k13]).reshape(3,1)
            k2 = np.array([k21, k22, k23]).reshape(3,1)
            
            # T11は3x3行列でなければならない
            T11[i,j,:,:] = np.outer(k1, np.conjugate(k1).T)
            T22[i,j,:,:] = np.outer(k2, np.conjugate(k2).T)
            Omega12[i,j,:,:] = np.outer(k1,np.conjugate(k2).T)
            
    result = np.zeros((x,y), dtype=np.complex)
    for i in range(x):
        for j in range(y):     
            Sigma = np.dot(np.dot(np.sqrt(T11[i,j,:,:]),Omega12[i,j,:,:]),np.sqrt(T22[i,j,:,:]))
            U, S, V = np.linalg.svd(Sigma, full_matrices=False)
            result[i,j] = np.dot(np.dot(np.conjugate(U[0]).T,Omega12[i,j,:,:]),V[0])
    return result

result = Complex_coherence(HH1,HV1,VV1,t_HH2,t_HV2,t_VV2,np.shape(HH1)[0],np.shape(HH1)[1])

In [ ]:
# コヒーレンスの高い地域を可視化
coh = 10*np.log10(abs(result)) -83.0 -32.0
coh = np.angle(np.array(coh, dtype=np.complex64))
coh[coh == -inf] = 0
coh = np.array(255*(coh-np.amin(coh))/(np.amax(coh)-np.amin(coh)),dtype="uint8")

img = cv2.flip(np.array(coh, dtype=np.float64), 1)
plt.imshow(np.array(img, dtype=np.float64))
plt.imsave('coherence_result.jpg', img, cmap = "jet")